In [9]:
# from ultralytics import YOLO
# from IPython.display import display, Image
# import cv2
# import numpy as np
# # from object_detection import ObjectDetection
# import math
# from sort import *

def dist(P, A, B):
    area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
    AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
    return ( area / AB )

f1_A=(388,821)
f1_B=(661,404) #distance 40

f2_A=(582, 876) #id 194
f2_B=(703, 415) #id 91 distance 30

f3_A=(782, 865) #id 60
f3_B=(743, 417) #id 30  distance 30

f4_A=(999, 866) #id 105
f4_B=(789, 417)  #id 85  distacne 30

f5_A=(1183, 850) #id 4
f5_B=(846, 429) #id 29 distance 30

# Initialize Object Detection
# od = ObjectDetection()
model = YOLO('yolov8n.pt')
cap = cv2.VideoCapture("los_angeles.mp4")

frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))	# 영상의 넓이(가로) 프레임
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))	# 영상의 높이(세로) 프레임
frame_size = (frameWidth, frameHeight)

print(frame_size)
# Initialize count
count = 0
center_points_prev_frame = []

danger_car_id=[]

tracking_objects = {}
track_id = 0

tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

while True:
    retval, frame = cap.read()
    count += 1
    if not(retval):
        break

    # Point current frame
    center_points_cur_frame = []

    # Detect objects on frame
    results = model(frame,stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cx = int((x1+x2) / 2)
            cy = int((y1+y2) / 2)
            center_points_cur_frame.append((cx, cy, 'Danger!'))
            #print("FRAME N°", count, " ", x, y, w, h)

            # cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            cv2.rectangle(frame, (x1, y1), (x2,y2), (0, 255, 0), 2)

    # Only at the beginning we compare previous and current frame
    if count <= 2:
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                if distance < 20:
                    tracking_objects[track_id] = pt
                    track_id += 1
    else:

        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()

        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            
            for pt in center_points_cur_frame_copy:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                # Update IDs position
                if distance < 20:
                    tracking_objects[object_id] = pt
                    object_exists = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                    continue

            # Remove IDs lost
            if not object_exists:
                tracking_objects.pop(object_id)

        # Add new IDs found
        for pt in center_points_cur_frame:
            tracking_objects[track_id] = pt
            track_id += 1

                
    for object_id, pt in tracking_objects.items():
        if dist((pt[0],pt[1]), f1_A, f1_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f1')
        
        if dist((pt[0],pt[1]), f2_A, f2_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f2')
            
        if dist((pt[0],pt[1]), f3_A, f3_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f3')
        
        if dist((pt[0],pt[1]), f4_A, f4_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f4')
        
        if dist((pt[0],pt[1]), f5_A, f5_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f5')
            
            
    for object_id, pt in tracking_objects.items():
        tmp= pt[2]+'['+str(object_id)+']'
        cv2.circle(frame, (pt[0],pt[1]), 5, (0, 0, 255), -1)
        cv2.putText(frame, tmp, (pt[0], pt[1] - 7), 0, 0.8, (0, 0, 255), 2)
        
        if (pt[2]=='Danger!'):
            danger_car_id.append(object_id)
            
                
    
    

    print("Tracking objects")
    print(tracking_objects)


    print("CUR FRAME LEFT PTS")
    print(center_points_cur_frame)

#     cv2.circle(frame, (1,1), 5, (0, 0, 255), -1)

    cv2.imshow("Frame", frame)

    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()

    key = cv2.waitKey(0)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


Ultralytics YOLOv8.0.26  Python-3.10.9 torch-1.11.0+cpu CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs



(1920, 1080)


0: 384x640 16 cars, 115.8ms
Speed: 0.8ms pre-process, 115.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Tracking objects
{}
CUR FRAME LEFT PTS
[(931, 536, 'Danger!'), (1021, 651, 'Danger!'), (763, 650, 'Danger!'), (1336, 979, 'Danger!'), (613, 477, 'Danger!'), (688, 454, 'Danger!'), (1271, 435, 'Danger!'), (440, 741, 'Danger!'), (569, 892, 'Danger!'), (882, 476, 'Danger!'), (943, 461, 'Danger!'), (859, 566, 'Danger!'), (754, 462, 'Danger!'), (1761, 636, 'Danger!'), (1116, 432, 'Danger!'), (844, 423, 'Danger!')]



0: 384x640 17 cars, 97.5ms
Speed: 0.0ms pre-process, 97.5ms inference, 9.6ms postprocess per image at shape (1, 3, 640, 640)


...

Tracking objects
{522: (1024, 613, 'f5'), 532: (486, 799, 'Danger!'), 581: (820, 794, 'Danger!'), 651: (958, 522, 'f5'), 664: (918, 653, 'f4'), 666: (641, 536, 'f2'), 670: (682, 680, 'Danger!'), 758: (722, 553, 'Danger!'), 833: (1136, 755, 'f5'), 837: (571, 1043, 'Danger!'), 843: (677, 472, 'f2'), 846: (878, 538, 'f4'), 863: (738, 481, 'f3'), 867: (1731, 553, 'Danger!'), 868: (1007, 844, 'f4'), 871: (1413, 501, 'Danger!'), 872: (1173, 457, 'Danger!')}
CUR FRAME LEFT PTS
[]
Tracking objects
{522: (1032, 620, 'f5'), 581: (826, 808, 'Danger!'), 651: (963, 525, 'Danger!'), 664: (925, 661, 'f4'), 666: (643, 544, 'f2'), 670: (684, 693, 'Danger!'), 758: (726, 558, 'f3'), 833: (1146, 770, 'f5'), 843: (680, 474, 'f2'), 846: (884, 544, 'Danger!'), 863: (742, 485, 'f3'), 871: (1422, 501, 'Danger!'), 872: (1166, 456, 'Danger!'), 874: (1706, 550, 'Danger!'), 875: (482, 820, 'Danger!'), 876: (812, 563, 'Danger!'), 877: (1225, 460, 'Danger!'), 878: (482, 822, 'Danger!'), 879: (1021, 863, 'f4'), 880: 

0: 384x640 18 cars, 1 bus, 2 trucks, 128.7ms
Speed: 0.0ms pre-process, 128.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 19 cars, 2 trucks, 134.6ms
Speed: 0.0ms pre-process, 134.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



Tracking objects
{522: (1035, 624, 'f5'), 581: (829, 815, 'Danger!'), 651: (966, 528, 'Danger!'), 664: (927, 666, 'f4'), 666: (645, 547, 'f2'), 670: (686, 699, 'Danger!'), 758: (728, 560, 'f3'), 833: (1153, 773, 'f5'), 843: (681, 477, 'f2'), 846: (888, 546, 'Danger!'), 863: (744, 484, 'f3'), 871: (1406, 496, 'Danger!'), 872: (1164, 455, 'Danger!'), 874: (1696, 547, 'Danger!'), 875: (481, 831, 'Danger!'), 876: (813, 566, 'Danger!'), 878: (481, 831, 'Danger!'), 879: (1023, 873, 'f4'), 882: (922, 476, 'f5')}
CUR FRAME LEFT PTS
[]
Tracking objects
{522: (1040, 629, 'f5'), 581: (832, 826, 'Danger!'), 651: (970, 529, 'Danger!'), 664: (931, 671, 'f4'), 666: (646, 550, 'f2'), 670: (687, 708, 'Danger!'), 758: (730, 563, 'f3'), 833: (1159, 780, 'f5'), 843: (682, 481, 'f2'), 846: (890, 548, 'Danger!'), 863: (746, 488, 'f3'), 871: (1402, 493, 'Danger!'), 872: (1163, 454, 'Danger!'), 874: (1687, 547, 'Danger!'), 875: (480, 841, 'Danger!'), 876: (816, 567, 'Danger!'), 878: (480, 841, 'Danger!'), 879

0: 384x640 19 cars, 1 bus, 1 truck, 112.6ms
Speed: 1.0ms pre-process, 112.6ms inference, 14.9ms postprocess per image at shape (1, 3, 640, 640)


Tracking objects
{522: (1042, 632, 'f5'), 581: (836, 837, 'Danger!'), 651: (972, 529, 'Danger!'), 664: (935, 675, 'f4'), 666: (646, 551, 'f2'), 670: (687, 714, 'Danger!'), 758: (732, 570, 'f3'), 833: (1166, 790, 'f5'), 843: (687, 480, 'f2'), 846: (893, 551, 'Danger!'), 863: (747, 490, 'f3'), 871: (1391, 493, 'Danger!'), 872: (1162, 454, 'Danger!'), 874: (1678, 543, 'Danger!'), 875: (479, 852, 'Danger!'), 876: (821, 571, 'Danger!'), 878: (479, 852, 'Danger!'), 879: (1031, 894, 'f4'), 882: (928, 478, 'Danger!'), 883: (812, 508, 'f4')}
CUR FRAME LEFT PTS
[(812, 508, 'Danger!')]


In [2]:
from ultralytics import YOLO
from ultralytics import YOLO
from IPython.display import display, Image
import cv2
import numpy as np
# from object_detection import ObjectDetection
import math
# from sort import *

In [3]:
from sort import *
 

In [8]:
# from ultralytics import YOLO
# from IPython.display import display, Image
# import cv2
# import numpy as np
# # from object_detection import ObjectDetection
# import math
# from sort import *

def dist(P, A, B):
    area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
    AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
    return ( area / AB )

f1_A=(388,821)
f1_B=(661,404) #distance 40

f2_A=(582, 876) #id 194
f2_B=(703, 415) #id 91 distance 30

f3_A=(782, 865) #id 60
f3_B=(743, 417) #id 30  distance 30

f4_A=(999, 866) #id 105
f4_B=(789, 417)  #id 85  distacne 30

f5_A=(1183, 850) #id 4
f5_B=(846, 429) #id 29 distance 30

# Initialize Object Detection
# od = ObjectDetection()
model = YOLO('yolov8n.pt')
cap = cv2.VideoCapture("los_angeles.mp4")

frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))	# 영상의 넓이(가로) 프레임
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))	# 영상의 높이(세로) 프레임
frame_size = (frameWidth, frameHeight)

print(frame_size)
# Initialize count
count = 0
center_points_prev_frame = []

danger_car_id=[]

tracking_objects = {}
track_id = 0

tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

while True:
    retval, frame = cap.read()
    count += 1
    if not(retval):
        break

    # Point current frame
    center_points_cur_frame = []

    # Detect objects on frame
    results = model(frame,stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cx = int((x1+x2) / 2)
            cy = int((y1+y2) / 2)
            center_points_cur_frame.append((cx, cy, 'Danger!'))
            #print("FRAME N°", count, " ", x, y, w, h)

            # cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            cv2.rectangle(frame, (x1, y1), (x2,y2), (0, 255, 0), 2)

    # Only at the beginning we compare previous and current frame
   

                
    for object_id, pt in tracking_objects.items():
        if dist((pt[0],pt[1]), f1_A, f1_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f1')
        
        if dist((pt[0],pt[1]), f2_A, f2_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f2')
            
        if dist((pt[0],pt[1]), f3_A, f3_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f3')
        
        if dist((pt[0],pt[1]), f4_A, f4_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f4')
        
        if dist((pt[0],pt[1]), f5_A, f5_B) < 30:
            tracking_objects[object_id]=(pt[0],pt[1],'f5')
            
            
    for object_id, pt in tracking_objects.items():
        tmp= pt[2]+'['+str(object_id)+']'
        cv2.circle(frame, (pt[0],pt[1]), 5, (0, 0, 255), -1)
        cv2.putText(frame, tmp, (pt[0], pt[1] - 7), 0, 0.8, (0, 0, 255), 2)
        
        if (pt[2]=='Danger!'):
            danger_car_id.append(object_id)
            
                
    
    

    print("Tracking objects")
    print(tracking_objects)


    print("CUR FRAME LEFT PTS")
    print(center_points_cur_frame)

#     cv2.circle(frame, (1,1), 5, (0, 0, 255), -1)

    cv2.imshow("Frame", frame)

    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()

    key = cv2.waitKey(0)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


(1920, 1080)


Ultralytics YOLOv8.0.26  Python-3.10.9 torch-1.11.0+cpu CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

0: 384x640 16 cars, 392.4ms
Speed: 9.2ms pre-process, 392.4ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


Tracking objects
{}
CUR FRAME LEFT PTS
[(931, 536, 'Danger!'), (1021, 651, 'Danger!'), (763, 650, 'Danger!'), (1336, 979, 'Danger!'), (613, 477, 'Danger!'), (688, 454, 'Danger!'), (1271, 435, 'Danger!'), (440, 741, 'Danger!'), (569, 892, 'Danger!'), (882, 476, 'Danger!'), (943, 461, 'Danger!'), (859, 566, 'Danger!'), (754, 462, 'Danger!'), (1761, 636, 'Danger!'), (1116, 432, 'Danger!'), (844, 423, 'Danger!')]



0: 384x640 17 cars, 174.7ms
Speed: 5.0ms pre-process, 174.7ms inference, 10.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 16 cars, 108.5ms
Speed: 0.0ms pre-process, 108.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Tracking objects
{}
CUR FRAME LEFT PTS
[(932, 537, 'Danger!'), (763, 654, 'Danger!'), (1021, 654, 'Danger!'), (612, 478, 'Danger!'), (1345, 983, 'Danger!'), (687, 455, 'Danger!'), (944, 460, 'Danger!'), (884, 475, 'Danger!'), (435, 749, 'Danger!'), (860, 568, 'Danger!'), (567, 906, 'Danger!'), (1265, 434, 'Danger!'), (755, 463, 'Danger!'), (1770, 631, 'Danger!'), (1730, 625, 'Danger!'), (1113, 432, 'Danger!'), (845, 424, 'Danger!')]
Tracking objects
{}
CUR FRAME LEFT PTS
[(932, 538, 'Danger!'), (687, 457, 'Danger!'), (946, 463, 'Danger!'), (764, 660, 'Danger!'), (612, 479, 'Danger!'), (563, 920, 'Danger!'), (1027, 656, 'Danger!'), (1354, 988, 'Danger!'), (1261, 433, 'Danger!'), (431, 756, 'Danger!'), (885, 476, 'Danger!'), (862, 572, 'Danger!'), (1110, 431, 'Danger!'), (1741, 623, 'Danger!'), (755, 467, 'Danger!'), (1413, 463, 'Danger!')]


...

Tracking objects
{}
CUR FRAME LEFT PTS
[(676, 514, 'Danger!'), (571, 548, 'Danger!'), (1100, 582, 'Danger!'), (623, 480, 'Danger!'), (764, 519, 'Danger!'), (1162, 823, 'Danger!'), (914, 509, 'Danger!'), (984, 595, 'Danger!'), (818, 478, 'Danger!'), (871, 451, 'Danger!'), (954, 769, 'Danger!'), (1362, 462, 'Danger!'), (953, 767, 'Danger!'), (708, 415, 'Danger!')]
Tracking objects
{}
CUR FRAME LEFT PTS
[(985, 597, 'Danger!'), (567, 551, 'Danger!'), (675, 516, 'Danger!'), (622, 482, 'Danger!'), (1115, 594, 'Danger!'), (915, 510, 'Danger!'), (872, 453, 'Danger!'), (1172, 837, 'Danger!'), (764, 523, 'Danger!'), (1581, 546, 'Danger!'), (819, 480, 'Danger!'), (961, 786, 'Danger!'), (960, 788, 'Danger!')]



0: 384x640 13 cars, 1 truck, 137.1ms
Speed: 1.0ms pre-process, 137.1ms inference, 6.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 13 cars, 1 truck, 148.5ms
Speed: 0.0ms pre-process, 148.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)



Tracking objects
{}
CUR FRAME LEFT PTS
[(676, 518, 'Danger!'), (1121, 597, 'Danger!'), (567, 556, 'Danger!'), (871, 455, 'Danger!'), (986, 599, 'Danger!'), (915, 511, 'Danger!'), (619, 482, 'Danger!'), (765, 525, 'Danger!'), (1178, 842, 'Danger!'), (820, 482, 'Danger!'), (1565, 542, 'Danger!'), (1371, 456, 'Danger!'), (967, 794, 'Danger!'), (966, 796, 'Danger!')]
Tracking objects
{}
CUR FRAME LEFT PTS
[(674, 521, 'Danger!'), (567, 558, 'Danger!'), (1125, 601, 'Danger!'), (988, 601, 'Danger!'), (916, 513, 'Danger!'), (1183, 846, 'Danger!'), (619, 485, 'Danger!'), (872, 455, 'Danger!'), (765, 528, 'Danger!'), (819, 481, 'Danger!'), (1542, 538, 'Danger!'), (970, 804, 'Danger!'), (969, 804, 'Danger!'), (1371, 453, 'Danger!')]


0: 384x640 14 cars, 127.9ms
Speed: 0.0ms pre-process, 127.9ms inference, 15.5ms postprocess per image at shape (1, 3, 640, 640)


Tracking objects
{}
CUR FRAME LEFT PTS
[(764, 530, 'Danger!'), (565, 560, 'Danger!'), (674, 521, 'Danger!'), (1137, 613, 'Danger!'), (990, 603, 'Danger!'), (619, 485, 'Danger!'), (917, 514, 'Danger!'), (874, 456, 'Danger!'), (1190, 854, 'Danger!'), (821, 482, 'Danger!'), (974, 814, 'Danger!'), (1357, 452, 'Danger!'), (1544, 535, 'Danger!'), (1530, 533, 'Danger!')]


In [1]:
import cv2
import numpy as np
from object_detection import ObjectDetection
import math

# Initialize Object Detection
od = ObjectDetection()

cap = cv2.VideoCapture("los_angeles.mp4")

frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))	# 영상의 넓이(가로) 프레임
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))	# 영상의 높이(세로) 프레임
frame_size = (frameWidth, frameHeight)

# Initialize count
count = 0
center_points_prev_frame = []

tracking_objects = {}
track_id = 0

while True:
    retval, frame = cap.read()
    count += 1
    if not(retval):
        break

    # Point current frame
    center_points_cur_frame = []

    # Detect objects on frame
    (class_ids, scores, boxes) = od.detect(frame)
    for box in boxes:
        (x, y, w, h) = box
        cx = int((x + x + w) / 2)
        cy = int((y + y + h) / 2)
        center_points_cur_frame.append((cx, cy))
        #print("FRAME N°", count, " ", x, y, w, h)

        # cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Only at the beginning we compare previous and current frame
    if count <= 2:
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                if distance < 20:
                    tracking_objects[track_id] = pt
                    track_id += 1
    else:

        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()

        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            for pt in center_points_cur_frame_copy:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                # Update IDs position
                if distance < 20:
                    tracking_objects[object_id] = pt
                    object_exists = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                    continue

            # Remove IDs lost
            if not object_exists:
                tracking_objects.pop(object_id)

        # Add new IDs found
        for pt in center_points_cur_frame:
            tracking_objects[track_id] = pt
            track_id += 1

    for object_id, pt in tracking_objects.items():
        cv2.circle(frame, pt, 5, (0, 0, 255), -1)
        cv2.putText(frame, str(object_id), (pt[0], pt[1] - 7), 0, 1, (0, 0, 255), 2)

    print("Tracking objects")
    print(tracking_objects)


    print("CUR FRAME LEFT PTS")
    print(center_points_cur_frame)


    cv2.imshow("Frame", frame)

    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()

    key = cv2.waitKey(0)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


Loading Object Detection
Running opencv dnn with YOLOv4
Tracking objects
{}
CUR FRAME LEFT PTS
[(571, 891), (437, 742), (1750, 635), (761, 648), (1018, 647), (928, 533), (856, 565), (612, 473), (881, 473), (1336, 980), (753, 463), (1118, 436), (942, 458), (1877, 605), (642, 435), (688, 450), (1268, 433), (1424, 466)]


In [10]:
import tensorflow as tf
import os

def get_mkl_enabled_flag():

    mkl_enabled = False
    major_version = int(tf.__version__.split(".")[0])
    minor_version = int(tf.__version__.split(".")[1])
    if major_version >= 2:
        if minor_version < 5:
            from tensorflow.python import _pywrap_util_port
        else:
            from tensorflow.python.util import _pywrap_util_port
            onednn_enabled = int(os.environ.get('TF_ENABLE_ONEDNN_OPTS', '0'))
        mkl_enabled = _pywrap_util_port.IsMklEnabled() or (onednn_enabled == 1)
    else:
        mkl_enabled = tf.pywrap_tensorflow.IsMklEnabled()
    return mkl_enabled

print ("We are using Tensorflow version", tf.__version__)
print("MKL enabled :", get_mkl_enabled_flag())



# Check available physical devices (GPUs)
physical_devices = tf.config.list_physical_devices('GPU')

if len(physical_devices) == 0:
    print("No GPU devices found.")
else:
    for device in physical_devices:
        print("GPU:", device)

# Check if Intel GPU is being used
intel_gpu_in_use = False
for device in physical_devices:
    if 'Intel' in tf.config.experimental.get_device_name(device):
        intel_gpu_in_use = True

print("Is Intel GPU being used?", intel_gpu_in_use)

ModuleNotFoundError: No module named 'tensorflow'

In [12]:
import tensorflow

ModuleNotFoundError: No module named 'tensorflow'